# PolicyEngine Itemization Bug Demo

This notebook demonstrates a bug where household 4428 itemizes despite worse tax outcomes.

In [ ]:
from policyengine_us import Microsimulation
from policyengine_us.model_api import *
import numpy as np

## Setup

Using household 4428 from the enhanced CPS 2024 dataset:

In [ ]:
DATASET = "hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5"
HOUSEHOLD_ID = 4428
YEAR = 2026

## Create Minimal Reform Stack

The bug occurs when CTC reform is applied after other reforms:

In [ ]:
# Minimal reform stack that triggers the bug
reform = Reform.from_dict({
    # Standard deduction increase
    "gov.irs.deductions.standard.amount.JOINT": {"2026-01-01": 48900},
    # Exemption elimination
    "gov.irs.income.exemption.amount": {"2026-01-01": 0},
    # CTC changes
    "gov.contrib.reconciliation.ctc.in_effect": {"2026-01-01": True},
    "gov.irs.credits.ctc.amount.base[0].amount": {"2026-01-01": 2200},
}, country_id="us")

## Demonstrate the Bug

In [ ]:
# Create simulation
sim = Microsimulation(reform=reform, dataset=DATASET)

# Find household
household_ids = sim.calculate("household_id", map_to="household", period=YEAR).values
idx = np.where(household_ids == HOUSEHOLD_ID)[0][0]

# Get values
itemizes = sim.calculate("tax_unit_itemizes", map_to="household", period=YEAR).values[idx]
tax_if_item = sim.calculate("tax_liability_if_itemizing", map_to="household", period=YEAR).values[idx]
tax_if_std = sim.calculate("tax_liability_if_not_itemizing", map_to="household", period=YEAR).values[idx]

print(f"Itemizes: {bool(itemizes)}")
print(f"Tax if itemizing: ${tax_if_item:,.2f}")
print(f"Tax if standard: ${tax_if_std:,.2f}")
print(f"Should itemize: {tax_if_item < tax_if_std}")
print(f"\nBUG: Household itemizes despite ${tax_if_item - tax_if_std:,.2f} worse outcome!")

## Verify Bug Disappears Without CTC Reform

In [ ]:
# Test without CTC reform
reform_no_ctc = Reform.from_dict({
    "gov.irs.deductions.standard.amount.JOINT": {"2026-01-01": 48900},
    "gov.irs.income.exemption.amount": {"2026-01-01": 0},
}, country_id="us")

sim_no_ctc = Microsimulation(reform=reform_no_ctc, dataset=DATASET)
itemizes_no_ctc = sim_no_ctc.calculate("tax_unit_itemizes", map_to="household", period=YEAR).values[idx]

print(f"Without CTC reform, itemizes: {bool(itemizes_no_ctc)}")
print("✓ Correctly takes standard deduction without CTC reform")